In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.apache.spark::spark-mllib:3.1.1`
import $ivy.`sh.almond::almond-spark:0.11.2`
import $ivy.`org.plotly-scala::plotly-almond:0.5.2`

import java.nio.file.{Paths, Files}
import scala.math.Ordered._
import scala.reflect.runtime.universe._

import org.apache.spark.sql.{DataFrame, SparkSession,Row}
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.{Matrix, Vector}
import org.apache.spark.ml.stat.{Correlation, Summarizer}
import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,OneHotEncoder,MinMaxScaler,Normalizer}
import org.apache.spark.ml.Pipeline

import plotly._, plotly.element._ , plotly.layout._ , plotly.Almond._ 

import org.apache.log4j.{Logger, Level}
Logger.getRootLogger.setLevel(Level.ERROR)
Logger.getRootLogger.setLevel(Level.FATAL)
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                               

import $ivy.$                                      


import java.nio.file.{Paths, Files}

import scala.math.Ordered._

import scala.reflect.runtime.universe._


import org.apache.spark.sql.{DataFrame, SparkSession,Row}

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.ml.linalg.{Matrix, Vector}

import org.apache.spark.ml.stat.{Correlation, Summarizer}

import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,OneHotEncoder,MinMaxScaler,Normalizer}

import org.apache.spark.ml.Pipeline


import plotly._, plotly.element._ , plotly.layout._ , plotly.Almond._ 


import org.apache.log4j.{Logger, Level}


In [2]:
val DATA_PATH_IN= "/tmp/sna.parquet"
val FEATURES_PATH= "data/features.txt"

println("Checking necessary files:")
if (Files.exists(Paths.get(FEATURES_PATH))) println("Found features") else println(f"ERROR! There is no data found on ${FEATURES_PATH}! Please run explore.ipynb first!") 
if (Files.exists(Paths.get(DATA_PATH_IN))) println("Found data source") else println(f"ERROR! There is no data found on ${DATA_PATH_IN}! Please run explore.ipynb first!") 

Checking necessary files:
Found features
Found data source


DATA_PATH_IN: String = "/tmp/sna.parquet"
FEATURES_PATH: String = "data/features.txt"

In [3]:
val spark=SparkSession.builder()
    .appName("Data Sources Practice")
    .config("spark.master", "local")
    .getOrCreate()

import spark.implicits._

val featuresNames=spark.sparkContext.objectFile[String](FEATURES_PATH).collect
val dfAssembled=spark.read.parquet(DATA_PATH_IN)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/08/10 19:46:55 WARN Utils: Your hostname, DESKTOP-G76NQH1 resolves to a loopback address: 127.0.1.1; using 172.17.97.97 instead (on interface eth0)
21/08/10 19:46:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/10 19:46:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark: SparkSession = org.apache.spark.sql.SparkSession@17406859
import spark.implicits._


featuresNames: Array[String] = Array(
  "instanceId_objectType_Post",
  "instanceId_objectType_Photo",
  "instanceId_objectType_Video",
  "audit_resourceType_8",
  "audit_resourceType_3",
  "audit_resourceType_7",
  "audit_resourceType_6",
  "audit_resourceType_14",
  "metadata_ownerType_GROUP_OPEN_OFFICIAL",
  "metadata_ownerType_GROUP_OPEN",
  "membership_status_A",
  "membership_status_P",
  "membership_status_I",
  "membership_status_M",
  "membership_status_Y",
  "membership_status_!",
  "membership_status_B",
  "membership_status_R",
  "membership_statusUpdateDate",
  "membership_joinDate",
  "membership_joinRequestDate",
  "auditweights_ageMs",
  "auditweights_closed",
  "auditweights_ctr_gender",
  "auditweights_ctr_high",
  "auditweights_ctr_negative",
  "auditweights_dailyRecency",
  "auditweights_feedOwner_RECOMMENDED_GROUP",
  "auditweights_feedStats",
  "auditweights_friendCommentFe

In [4]:
val splits:Array[DataFrame] = dfAssembled
    //.select($"features",$"label")
    .randomSplit(Array(0.7, 0.3), seed = 21)

val scaler=new MinMaxScaler().setInputCol("features_comb").setOutputCol("features").fit(splits(0))
val dfTrain=scaler.transform(splits(0))
val dfTest=scaler.transform(splits(1))

val label="label"
val features="features"

splits: Array[DataFrame] = Array(
  [instanceId_userId: int, features_mean: vector ... 5 more fields],
  [instanceId_userId: int, features_mean: vector ... 5 more fields]
)
scaler: org.apache.spark.ml.feature.MinMaxScalerModel = MinMaxScalerModel: uid=minMaxScal_319eee0e56f5, numFeatures=82, min=0.0, max=1.0
dfTrain: DataFrame = [instanceId_userId: int, features_mean: vector ... 6 more fields]
dfTest: DataFrame = [instanceId_userId: int, features_mean: vector ... 6 more fields]
label: String = "label"
features: String = "features"

# Models
## RandomForestClassifier

In [9]:
import org.apache.spark.ml.classification.RandomForestClassifier

val rf = new RandomForestClassifier()
  .setLabelCol(label)
  .setFeaturesCol(features)
  .setNumTrees(20)

val rfModel = rf.fit(dfTrain)
val dfrfOutput = rfModel.transform(dfTest)

import org.apache.spark.ml.classification.RandomForestClassifier


rf: RandomForestClassifier = rfc_83c1b29327ff
rfModel: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel: uid=rfc_83c1b29327ff, numTrees=20, numClasses=2, numFeatures=81
dfrfOutput: DataFrame = [instanceId_userId: int, features_mean: vector ... 9 more fields]

## NaiveBayes

In [10]:
import org.apache.spark.ml.classification.NaiveBayes

val nb = new NaiveBayes()
  .setLabelCol(label)
  .setFeaturesCol(features)

val nbModel = nb.fit(dfTrain)
val dfnbOutput = nbModel.transform(dfTest)

import org.apache.spark.ml.classification.NaiveBayes


nb: NaiveBayes = nb_43e4612db691
nbModel: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel: uid=nb_43e4612db691, modelType=multinomial, numClasses=2, numFeatures=81
dfnbOutput: DataFrame = [instanceId_userId: int, features_mean: vector ... 9 more fields]

## GBTClassifier

In [5]:
import org.apache.spark.ml.classification.GBTClassifier

val gbt = new GBTClassifier()
  .setLabelCol(label)
  .setFeaturesCol(features)
  .setMaxIter(20)

val gbtModel = gbt.fit(dfTrain)
val dfGbtOutput = gbtModel.transform(dfTest).cache()

import org.apache.spark.ml.classification.GBTClassifier


gbt: GBTClassifier = gbtc_f04ac639e166
gbtModel: org.apache.spark.ml.classification.GBTClassificationModel = GBTClassificationModel: uid = gbtc_f04ac639e166, numTrees=20, numClasses=2, numFeatures=82
dfGbtOutput: org.apache.spark.sql.Dataset[Row] = [instanceId_userId: int, features_mean: vector ... 9 more fields]

In [12]:
def metrics(name:String, df:DataFrame) = {
    val res=df.groupBy("label", "prediction")
        .agg(count("prediction").as("total"))
        .select($"total").as[Long].collect
    val Array(fn,tn,tp,fp) = res.map(_.toDouble)
    val precision = tp / (tp + fp) // Precision (Positive Predictive Value)
    val recall = tp / (tp + fn) // Recall (True Positive Rate)
    val f1 = (2 * precision * recall) / (precision + recall)
    f"${name}: PRECISION=${precision}%.2f, RECALL=${recall}%.2f, F1 score=${f1}%.2f"
}

println(metrics("NB", dfnbOutput))
println(metrics("RF", dfrfOutput))
println(metrics("GBT", dfGbtOutput))

21/08/10 02:54:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/10 02:54:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


NB: PRECISION=0.95, RECALL=0.00, F1 score=0.00
RF: PRECISION=0.89, RECALL=0.26, F1 score=0.40
GBT: PRECISION=0.85, RECALL=0.70, F1 score=0.77


defined function metrics

The best model is GBTClassifier, let's use it!

# Feature Importance

In [ ]:
import org.apache.spark.ml.classification.GBTClassificationModel
val importances = gbtModel // this would be your trained model
  .asInstanceOf[GBTClassificationModel]
  .featureImportances.toSparse
val featureImportances = ((for (idx<-importances.indices) yield (featuresNames(idx))) zip importances.values).sortWith(_._2 > _._2).take(15)
val (x,y) = featureImportances.unzip

In [7]:
Bar(x.toList,y.toList).plot(title = "Features Importance")

res6: String = "plot-2094044100"

Well, the best feature is one from numerical features, I suppose it derived from auditweight analyse.  But next three places taken by newly designed features. 

In [ ]:
def plotBox(df:DataFrame, feature:String, title:String) = {
    val (x,y) = df.select($"label",col(feature))
        .collect
        .map(r=>(r(0).toString, r(1).toString.toDouble))
        .toList.unzip
    Box(x,y,orientation = Orientation.Horizontal).plot(title = title)
}
plotBox(dfGbtOutput, "ddiff_max", "Distribution of ddiff_max")

There is a lot of data by plotly, so I've saved diagrams to reduce notebook size.

![](./images/ddiff_max.png)

ddiff_max with median 7 anf 15 for both classes accordingly, clearly shows significant difference between classes. Churned users (class 1) have more then twice less time difference between activities. It is counterintuitive and go against common sense. It should be vice versa, and large laps in user activity must be consistent with greater chance user to churn. 

I have only one explanation - we have observations for a short period of time, so large gaps inside that period corresponds to small last gap (less then 23 days), so user counts as not churned, but who knows what has happen later. 

So, we have to make analysis with uncertain target and unclear criteries.  The probability of getting an appropriate result is vanishingly small.

In [ ]:
plotBox(dfGbtOutput, "activity_sum", "Distribution of activity_sum")

![](./images/activity_sum.png)

Churned users are much less active, so this feature make sense!

In [ ]:
plotBox(dfGbtOutput, "activity_mean", "Distribution of activity_mean")

![](./images/activity_mean.png)

No much difference between groups here, so I can't make any meaningful conclusions why this parameter so important for the model.